In [4]:
!mkdir -p /content/paderborn_local
!unzip -q "/content/drive/MyDrive/Padeborn_Dataset/padeborn.zip" -d /content/paderborn_local

In [12]:
dataset_path = "/content/paderborn_local"

In [13]:
import os
import numpy as np
import scipy.io
import scipy.stats
import pandas as pd
from scipy.fft import fft
from sklearn.model_selection import train_test_split

def extract_features(signal, segment_size=1024):
    num_segments = len(signal) // segment_size
    features = []
    for i in range(num_segments):
        seg = signal[i * segment_size : (i + 1) * segment_size]
        f_vals = np.abs(fft(seg)[:segment_size // 2])

        t_kurt = scipy.stats.kurtosis(seg)
        f_p2p = np.ptp(f_vals)
        f_skew = scipy.stats.skew(f_vals)
        t_p2p = np.ptp(seg)
        t_std = np.std(seg)
        f_rms = np.sqrt(np.mean(f_vals**2))
        f_crest = np.max(f_vals) / (f_rms + 1e-9)
        f_kurt = scipy.stats.kurtosis(f_vals)

        features.append([t_kurt, f_p2p, f_skew, t_p2p, t_std, f_rms, f_crest, f_kurt])
    return features

def get_signal(file_path):
    try:
        mat = scipy.io.loadmat(file_path)
        key = [k for k in mat.keys() if not k.startswith('_')][0]
        y_data = mat[key][0, 0]['Y']
        for i in range(y_data.shape[1]):
            if 'vibration_1' in y_data[0, i]['Name'][0]:
                return y_data[0, i]['Data'].flatten()
        return None
    except:
        return None

def build_dataset(base_path):
    data_rows = []
    for root, _, files in os.walk(base_path):
        for f in files:
            if f.endswith(".mat"):
                label = -1
                if "K00" in f: label = 0
                elif "KA" in f: label = 1
                elif "KI" in f: label = 2
                elif "KB" in f: label = 3

                if label == -1: continue

                sig = get_signal(os.path.join(root, f))
                if sig is not None:
                    rpm = 900 if "N09" in f else 1500
                    feats = extract_features(sig)
                    for ft in feats:
                        data_rows.append(ft + [rpm, label])

    cols = [
        'vibration_1_time_kurtosis', 'vibration_1_freq_peak_to_peak',
        'vibration_1_freq_skewness', 'vibration_1_time_peak_to_peak',
        'vibration_1_time_std_dev', 'vibration_1_freq_rms',
        'vibration_1_freq_crest_factor', 'vibration_1_freq_kurtosis',
        'rpm', 'label'
    ]
    return pd.DataFrame(data_rows, columns=cols)

df = build_dataset(dataset_path)

if not df.empty:
    train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
    train_df.to_csv('/content/drive/MyDrive/Padeborn_Dataset/train_fft_quantum.csv', index=False)
    test_df.to_csv('/content/drive/MyDrive/Padeborn_Dataset/test_fft_quantum.csv', index=False)
    print(f"Success! Train: {len(train_df)} | Test: {len(test_df)}")
else:
    print("Check path or file naming.")

Success! Train: 513090 | Test: 128273


In [17]:
import pandas as pd
import os


train_path = '/content/drive/MyDrive/Padeborn_Dataset/train_fft_quantum.csv'
test_path = '/content/drive/MyDrive/Padeborn_Dataset/test_fft_quantum.csv'

if os.path.exists(train_path) and os.path.exists(test_path):
    print("✅ Arquivos encontrados com sucesso!")


    df_train = pd.read_csv(train_path)
    df_test = pd.read_csv(test_path)


    print(f"\n--- Resumo do Dataset de Treino ---")
    print(f"Total de amostras: {len(df_train)}")
    print(f"Colunas: {df_train.columns.tolist()}")


    print("\n--- Distribuição por Classe (Label) ---")

    print(df_train['label'].value_counts().sort_index())


    print("\n--- Distribuição por RPM ---")
    print(df_train['rpm'].value_counts())


    print("\n--- Primeiras 5 linhas ---")
    print(df_train.head())
else:
    print("❌ Erro: Os arquivos não foram encontrados. Verifique o caminho do salvamento.")

✅ Arquivos encontrados com sucesso!

--- Resumo do Dataset de Treino ---
Total de amostras: 513090
Colunas: ['vibration_1_time_kurtosis', 'vibration_1_freq_peak_to_peak', 'vibration_1_freq_skewness', 'vibration_1_time_peak_to_peak', 'vibration_1_time_std_dev', 'vibration_1_freq_rms', 'vibration_1_freq_crest_factor', 'vibration_1_freq_kurtosis', 'rpm', 'label']

--- Distribuição por Classe (Label) ---
label
0     96162
1    192279
2    176519
3     48130
Name: count, dtype: int64

--- Distribuição por RPM ---
rpm
1500    384954
900     128136
Name: count, dtype: int64

--- Primeiras 5 linhas ---
   vibration_1_time_kurtosis  vibration_1_freq_peak_to_peak  \
0                   0.925965                      53.044153   
1                   2.433260                      82.733287   
2                   1.964131                      34.019643   
3                   4.445548                      42.423530   
4                   6.532173                      44.507963   

   vibration_1_freq

In [18]:
import pandas as pd


df = pd.read_csv('/content/drive/MyDrive/Padeborn_Dataset/train_fft_quantum.csv')


samples_per_class = 5000


balanced_list = []

for label in df['label'].unique():

    subset = df[df['label'] == label]


    n_samples = min(len(subset), samples_per_class)


    balanced_subset = subset.sample(n=n_samples, random_state=42)
    balanced_list.append(balanced_subset)


df_balanced = pd.concat(balanced_list).sample(frac=1).reset_index(drop=True)


df_balanced.to_csv('train_balanced_20k.csv', index=False)

print("--- Novo Dataset de Treino Quântico ---")
print(f"Total de amostras: {len(df_balanced)}")
print("\nDistribuição por Classe:")
print(df_balanced['label'].value_counts().sort_index())
print("\nDistribuição por RPM:")
print(df_balanced['rpm'].value_counts())

--- Novo Dataset de Treino Quântico ---
Total de amostras: 20000

Distribuição por Classe:
label
0    5000
1    5000
2    5000
3    5000
Name: count, dtype: int64

Distribuição por RPM:
rpm
1500    15013
900      4987
Name: count, dtype: int64


In [19]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np


df_balanced = pd.read_csv('train_balanced_20k.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Padeborn_Dataset/test_fft_quantum.csv') # Usamos o teste original para validação real


feature_cols = [
    'vibration_1_time_kurtosis', 'vibration_1_freq_peak_to_peak',
    'vibration_1_freq_skewness', 'vibration_1_time_peak_to_peak',
    'vibration_1_time_std_dev', 'vibration_1_freq_rms',
    'vibration_1_freq_crest_factor', 'vibration_1_freq_kurtosis'
]


scaler = MinMaxScaler(feature_range=(0, np.pi))

X_train_scaled = scaler.fit_transform(df_balanced[feature_cols])
y_train = df_balanced['label'].values


df_test_small = df_test.sample(n=2000, random_state=42)
X_test_scaled = scaler.transform(df_test_small[feature_cols])
y_test = df_test_small['label'].values


X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)


train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=32, shuffle=False)

print(f"Dados prontos!")
print(f"X_train shape: {X_train_tensor.shape}")
print(f"Gama de valores após scaler: {X_train_tensor.min().item():.2f} a {X_train_tensor.max().item():.2f}")

Dados prontos!
X_train shape: torch.Size([20000, 8])
Gama de valores após scaler: 0.00 a 3.14


In [20]:
import pennylane as qml
from pennylane import numpy as pnp
import torch.nn as nn


n_qubits = 8
dev = qml.device("default.qubit", wires=n_qubits)


@qml.qnode(dev, interface="torch")
def quantum_circuit(inputs, weights):

    qml.AngleEmbedding(inputs, wires=range(n_qubits))


    qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))


    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]


class HybridQuantumModel(nn.Module):
    def __init__(self, n_layers=3):
        super().__init__()

        weight_shapes = {"weights": (n_layers, n_qubits, 3)}


        self.clayer_1 = qml.qnn.TorchLayer(quantum_circuit, weight_shapes)


        self.fc = nn.Linear(n_qubits, 4)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.clayer_1(x)
        x = self.fc(x)
        return self.softmax(x)


model = HybridQuantumModel(n_layers=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.NLLLoss()

print(model)

HybridQuantumModel(
  (clayer_1): <Quantum Torch Layer: func=quantum_circuit>
  (fc): Linear(in_features=8, out_features=4, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [24]:
@qml.qnode(dev, interface="torch")
def quantum_circuit_reupload(inputs, weights):

    for i in range(weights.shape[0]):

        qml.AngleEmbedding(inputs, wires=range(n_qubits))
        qml.StronglyEntanglingLayers(weights[i:i+1], wires=range(n_qubits))

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

class QuantumTurboModel(nn.Module):
    def __init__(self, n_layers=4):
        super().__init__()

        weight_shapes = {"weights": (n_layers, n_qubits, 3)}
        self.quantum_layer = qml.qnn.TorchLayer(quantum_circuit_reupload, weight_shapes)

        self.classical_head = nn.Sequential(
            nn.Linear(8, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 4)
        )
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        q_out = self.quantum_layer(x)
        return self.softmax(self.classical_head(q_out))

model_v3 = QuantumTurboModel(n_layers=4).to(device)
optimizer = torch.optim.Adam(model_v3.parameters(), lr=0.01)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=3, factor=0.5)

In [26]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler


df_train = pd.read_csv('train_balanced_20k.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Padeborn_Dataset/test_fft_quantum.csv').sample(2000, random_state=42)

feature_cols = [
    'vibration_1_time_kurtosis', 'vibration_1_freq_peak_to_peak',
    'vibration_1_freq_skewness', 'vibration_1_time_peak_to_peak',
    'vibration_1_time_std_dev', 'vibration_1_freq_rms',
    'vibration_1_freq_crest_factor', 'vibration_1_freq_kurtosis'
]

X_train_raw = df_train[feature_cols].values
X_test_raw = df_test[feature_cols].values


std_scaler = StandardScaler()
X_train_std = std_scaler.fit_transform(X_train_raw)
X_test_std = std_scaler.transform(X_test_raw)


mm_scaler = MinMaxScaler(feature_range=(0, np.pi))
X_train_final = mm_scaler.fit_transform(X_train_std)
X_test_final = mm_scaler.transform(X_test_std)


X_train_tensor = torch.tensor(X_train_final, dtype=torch.float32)
y_train_tensor = torch.tensor(df_train['label'].values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_final, dtype=torch.float32)
y_test_tensor = torch.tensor(df_test['label'].values, dtype=torch.long)


train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=32, shuffle=False)

print("Dados normalizados com sucesso!")

Dados normalizados com sucesso!


In [27]:
history_v3 = train_model(
    model=model_v3,
    train_loader=train_loader,
    test_loader=test_loader,
    optimizer=optimizer,
    criterion=criterion,
    epochs=15
)

Epoch 1/15: 100%|██████████| 625/625 [01:54<00:00,  5.47it/s, loss=0.9908, acc=56.69%]



Epoch 1 Finalizada - Test Acc: 51.30% | Avg Loss: 0.9702



Epoch 2/15: 100%|██████████| 625/625 [01:49<00:00,  5.72it/s, loss=1.1265, acc=57.78%]



Epoch 2 Finalizada - Test Acc: 53.65% | Avg Loss: 0.9413



Epoch 3/15: 100%|██████████| 625/625 [01:26<00:00,  7.25it/s, loss=1.3294, acc=58.20%]


...


Epoch 13 Finalizada - Test Acc: 56.90% | Avg Loss: 0.9033



Epoch 14/15: 100%|██████████| 625/625 [01:24<00:00,  7.42it/s, loss=0.9174, acc=60.15%]



Epoch 14 Finalizada - Test Acc: 54.90% | Avg Loss: 0.9002



Epoch 15/15: 100%|██████████| 625/625 [01:24<00:00,  7.42it/s, loss=0.8580, acc=60.12%]



Epoch 15 Finalizada - Test Acc: 55.65% | Avg Loss: 0.9041

